In [ ]:
!pip install dnspython

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

#PostgreSQL access data
host = "bda_gr4_database"
port = "5432"
database = "domainanalysis"
user = "postgres"
password = "postgres"

# PostgreSQL connection url
connection = f"jdbc:postgresql://{host}:{port}/{database}"

# Create a Spark session
spark = SparkSession.builder \
    .appName("ETL_DomainAnaylsis") \
    .getOrCreate()

# Read data from the database
domains_df = spark.read \
                .format("jdbc") \
                .option("url", connection) \
                .option("dbtable", "domain") \
                .option("user", user) \
                .option("password", password) \
                .load()

# Display the data frame
# domains_df.show()

In [ ]:
import dns
from dns import resolver
import requests
from urllib.parse import urlparse


def getARecords(domain):
    try:
        result = dns.resolver.resolve(domain, 'A')
        return list(map(lambda ipval: ipval.to_text(), result))
        #for ipval in result:
        #    return ipval.to_text()
    except dns.resolver.NXDOMAIN:
        return "Domain not available"
    except dns.resolver.NoAnswer:
        return "No Answer"
    except dns.resolver.NoNameservers:
        return "No Nameservers"
    except dns.resolver.Timeout:
        return "Timeout"


def getMXRecords(domain):
    try:
        result = dns.resolver.resolve(domain, 'MX')
        return list(map(lambda mail: mail.exchange.to_text(), result))
        #result = dns.resolver.resolve(domain, 'MX')
        #for mail in result:
        #return mail.exchange.to_text()
    except dns.resolver.NXDOMAIN:
        return list("Domain not available")
    except dns.resolver.NoAnswer:
        return list("No Answer")
    except dns.resolver.NoNameservers:
        return list("No Nameservers")
    except dns.exception.Timeout:
        return list("Timeout")

def getRedirectUrl(domain):
    try:
        response = requests.get("http://" + domain, timeout=5)
        url = urlparse(response.url).netloc
        url = ('.'.join(url.split('.')[-2:]))
        return url
    except requests.exceptions.ConnectionError:
        return "Connection Error"
    except requests.exceptions.ReadTimeout:
        return "Read Timeout"
    except requests.exceptions.TooManyRedirects:
        return "Too Many Redirects"

def getStatusCodeUrl(domain):
    try:
        response = requests.get("http://" + domain, timeout=5)
        return response.status_code
    except requests.exceptions.ConnectionError:
        return "Connection Error"
    except requests.exceptions.ReadTimeout:
        return "Read Timeout"
    except requests.exceptions.TooManyRedirects:
        return "Too Many Redirects"

In [ ]:
udf_getARecords = udf(getARecords, ArrayType(StringType()))
udf_getMXRecords = udf(getMXRecords, ArrayType(StringType()))
udf_getRedirectUrl = udf(getRedirectUrl, ArrayType(StringType()))
udf_getStatusCodeUrl = udf(getStatusCodeUrl, ArrayType(StringType()))

In [ ]:
domains_df = domains_df.withColumn("a_record_checked", udf_getARecords("top_level_domain"))
domains_df = domains_df.withColumn("mx_record_checked", udf_getMXRecords("top_level_domain"))
domains_df = domains_df.withColumn("redirection", udf_getRedirectUrl("top_level_domain"))
domains_df = domains_df.withColumn("status_code", udf_getStatusCodeUrl("top_level_domain"))

In [ ]:
domains_df.show(20)

In [ ]:
# Count the values of each column with descending sort and show it
# domains_df.groupBy("Top-Level-Domain").count().sort(desc("count")).show(20, False)
# domains_df.groupBy("MX-Record").count().sort(desc("count")).show(20, False)
# domains_df.groupBy("A-Record").count().sort(desc("count")).show(20, False)
# domains_df.groupBy("A-Record_Checked").count().sort(desc("count")).show(20, False)
# domains_df.groupBy("MX-Record_Checked").count().sort(desc("count")).show(20, False)
# domains_df.groupBy("Redirection").count().sort(desc("count")).show(20, False)

In [ ]:
domains_df.withColumn('mx_record_count', explode(col('mx_record'))) \
        .groupBy('mx_record_count') \
        .count() \
        .sort(desc("count"))

domains_df.withColumn('mx_record_checked_count', explode(col('mx_record_checked'))) \
        .groupBy('mx_record_checked_count') \
        .count() \
        .sort(desc("count"))

domains_df.withColumn('a_record_count', explode(col('a_record'))) \
        .groupBy('a_record_count') \
        .count() \
        .sort(desc("count"))

domains_df.withColumn('a_record_checked', explode(col('a_record_checked'))) \
        .groupBy('a_record_checked') \
        .count() \
        .sort(desc("count"))

domains_df = domains_df.drop('mx_record').drop('a_record')

In [ ]:
# Write the data frame to the PostgreSQL database
domains_df.write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "domain_enhanced") \
    .option("user", user) \
    .option("password", password) \
    .mode("append") \
    .save()